In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
work_dir="datasets/tcga/BRCA/mirna_allsamples/"
#working_dir="../cancers/breast/"
os.chdir(work_dir)

## Survival

In [ ]:
df_files = pd.read_csv("files.dat", sep=",")
df_files.info()

In [ ]:
bins = np.linspace(0,100,20)
avg = df_files["cases.0.diagnoses.0.age_at_diagnosis"].mean(skipna=True)
df_files["age_at_diagnosis"]=pd.cut(df_files["cases.0.diagnoses.0.age_at_diagnosis"].fillna(avg)/365., bins=bins, labels = (bins[1:]+bins[:-1])/2).__array__()

In [ ]:
df_files["gender"] = (df_files["cases.0.demographic.gender"]=="male").astype(int)

In [ ]:
for letter in ["a", "b", "c"]:
    for (old, new) in zip(["stage %s%s"%(i,letter) for i in ["i", "ii", "iii", "iv"]],["stage %s"%i for i in ["i", "ii", "iii", "iv"]]):
        df_files.replace(old, new, inplace=True)
df_files["cases.0.diagnoses.0.tumor_stage"].unique()

In [ ]:
df_files["tumor_stage"]=df_files["cases.0.diagnoses.0.tumor_stage"]
for i,stage in enumerate(["stage i", "stage ii", "stage iii", "stage iv", "stage v", "stage x"]):
    df_files["tumor_stage"].replace(stage, i+1, inplace=True)

In [ ]:
#0 = Alive
df_files["vital_status"]=(df_files["cases.0.demographic.vital_status"]=="Dead").astype(int)

In [ ]:
def get_survival(case):
    if case["cases.0.demographic.vital_status"] == 1:
        return case["cases.0.demographic.days_to_death"]
    else:
        return case["cases.0.diagnoses.0.days_to_last_follow_up"]

df_files["days_survival"] = df_files.apply(get_survival,1)

## Univariate

In [ ]:
import lifelines
import sys
sys.path.append("/home/jovyan/work/phd/")
from survival import fit_cox, add_group_to_subset, save_plot
from topicpy.hsbmpy import get_file
from scipy.stats import pearsonr

In [ ]:
l=1

algorithm = "trisbm"
kind = "keyword", "metadatum"
#kind = "word", "topic"

In [ ]:
df_clusters = pd.read_csv("trisbm/trisbm_level_1_%s-dist.csv"%kind[1],index_col=1).drop("i_doc",1)

In [ ]:
df_topics = pd.read_csv("%s/%s_level_%d_%s-dist.csv"%(algorithm,algorithm,l,kind[1]))
df_metadata = pd.read_csv("files.dat", index_col=0).dropna(how='all', axis=0)
for letter in ["a", "b", "c"]:
    for (old, new) in zip(["stage %s%s"%(i,letter) for i in ["i", "ii", "iii", "iv"]],["stage %s"%i for i in ["i", "ii", "iii", "iv"]]):
        df_metadata.replace(old, new, inplace=True)

df_topics.set_index('doc', inplace=True)
df_topics.insert(0,'stage','')
df_topics.insert(0,'cohort','')
df_topics.drop('i_doc', axis=1, inplace=True)

labels=["cases.0.diagnoses.0.tumor_stage","Subtype_Selected"]
label = labels[0]

if label not in df_files.columns:
    raise AttributeError(f"{label} not valid. Available: {df_files.columns}")
for letter in ["a", "b"]:
    for (old, new) in zip(["stage %s%s"%(i,letter) for i in ["i", "ii", "iii", "iv"]],["stage %s"%i for i in ["i", "ii", "iii", "iv"]]):
        df_files.replace(old, new, inplace=True)
for sample in df_topics.index.values:
    file_data = get_file(sample,df_metadata)
    if file_data is None:
        continue
    if (file_data[labels[1]]=="nan"):
        name = "A"
    else:
        name = "B"
    df_topics.at[sample,'stage']=("%s"%(file_data[labels[0]]))
    df_topics.at[sample,'cohort']=("%s"%(name))

mask = df_topics["stage"]!="not reported"
mask = (mask) & (df_topics["stage"]!="")
mask = (mask) & (df_topics["stage"]!="stage x")
mask = (mask) & (df_topics["stage"]!="nan")
df_topic_stage_gb = df_topics[mask].groupby(["cohort", "stage"])

df_topic_stage = df_topics[mask]

color_generator=(c for c in sns.palettes.color_palette(n_colors=20, palette=None))
df_topic_stage = df_topic_stage_gb.mean()

for topic in df_topic_stage.columns:
    fig, axs = plt.subplots(1,3,figsize=(45,15))
    color = next(color_generator)
    X = []
    for ((i,dataset), ax) in zip(enumerate(np.unique([idx[0] for idx in df_topic_stage.index])), axs[:2]):
        idxs = int(round(df_topic_stage.index.shape[0]/2))
        idx = df_topic_stage.index[i*idxs:(i+1)*idxs]
        
        df_topic_stage_dat = df_topic_stage.reindex(index=idx, columns = [topic])
        mean = df_topic_stage_dat.mean(0).values
        df_topic_stage_dat=df_topic_stage_dat.subtract(mean,1)
        #sigma = df_topic_stage_dat.std(0).values[0]
        #df_topic_stage_dat=df_topic_stage_dat.divide(sigma,1)
        df_topic_stage_dat.plot(ax=ax, marker="o",ms=55, lw=20, c=color, legend=False)
        
        x = ax.lines[0].get_xdata()
        y = ax.lines[0].get_ydata()
        err = (df_topic_stage_dat.std(0)).values.ravel()
        ax.fill_between(ax.lines[0].get_xdata(), y-err, y+err, color="gray", alpha=0.8)
        
        
        labs = np.sort(df_topics[mask]["stage"].unique())
        ax.set_xticks(x)
        ax.set_xticklabels(labels=labs, rotation=45)
        ax.tick_params(labelsize=25)
        ax.set_title(dataset, fontsize=25)
        ax.set_xlabel("")
        
        X.append(y)
        
    axs[1].legend(fontsize=35, ncol=3)
    axs[0].set_ylabel("$Centered <P(%s|stage)>$"%kind[1], fontsize=35)
    
    ax = axs[2]
    ax.scatter(*X, s=55**2, color=color)
    for x,y, text in zip(X[0],X[1],["stage i","stage ii","stage iii","stage iv"]):
        ax.annotate(text, (x,y), size=35)
    ax.tick_params(labelsize=25)
    ax.set_title("A vs B,\n Pearson {:.3f}".format(pearsonr(X[0],X[1])[0]), fontsize=25)
    ax.set_xlabel("")
    ax.set_xlabel("$Centered <P(%s|stage,A)>$"%kind[1], fontsize=35)
    ax.set_ylabel("$Centered <P(%s|stage,B)>$"%kind[1], fontsize=35)
    ax.ticklabel_format(style="sci", useOffset=True)

    fig.savefig(f"{kind[1]}_A_B_{topic}.pdf")
    

    plt.show()

In [ ]:
df_topics = pd.read_csv("%s/%s_level_%d_%s-dist.csv"%(algorithm,algorithm,l,kind[1]))

df_topics.set_index('doc', inplace=True)
df_topics.insert(0,'stage','')
df_topics.drop('i_doc', axis=1, inplace=True)

labels=["cases.0.diagnoses.0.tumor_stage"]
label = labels[0]

if label not in df_files.columns:
    raise AttributeError(f"{label} not valid. Available: {df_files.columns}")
for letter in ["a", "b"]:
    for (old, new) in zip(["stage %s%s"%(i,letter) for i in ["i", "ii", "iii", "iv"]],["stage %s"%i for i in ["i", "ii", "iii", "iv"]]):
        df_files.replace(old, new, inplace=True)
for sample in df_topics.index.values:
    file_data = get_file(sample,df_metadata)
    if file_data is None:
        continue
    df_topics.at[sample,'stage']=("%s"%(file_data[labels[0]]))

mask = df_topics["stage"]!="not reported"
mask = (mask) & (df_topics["stage"]!="")
mask = (mask) & (df_topics["stage"]!="stage x")
mask = (mask) & (df_topics["stage"]!="nan")
df_topic_stage_gb = df_topics[mask].groupby(["stage"]).mean()

color_generator=(c for c in sns.palettes.color_palette(n_colors=20, palette=None))

In [ ]:
for topic in df_topic_stage_gb.columns:
    fig, ax = plt.subplots(1,1,figsize=(30,15))
    color = next(color_generator)


    df_topic_stage_dat = df_topic_stage_gb[topic]
    mean = df_topic_stage_dat.mean()
    df_topic_stage_dat=df_topic_stage_dat.subtract(mean,1)
    #sigma = df_topic_stage_dat.std(0).values[0]
    #df_topic_stage_dat=df_topic_stage_dat.divide(sigma,1)
    df_topic_stage_dat.plot(ax=ax, marker="o",ms=55, lw=20, c=color, legend=False)

    x = ax.lines[0].get_xdata()
    y = ax.lines[0].get_ydata()
    err = (df_topic_stage_dat.max()-df_topic_stage_dat.min())/2
    ax.fill_between(ax.lines[0].get_xdata(), y-err, y+err, color="gray", alpha=0.8)

    labs = np.sort(df_topics[mask]["stage"].unique())
    ax.set_xticks(x)
    ax.set_xticklabels(labels=labs, rotation=45)
    ax.tick_params(labelsize=35)
    ax.set_title(topic+"\nPearson:"+"{0:.3f}".format(pearsonr(x,y)[0]), fontsize=35)
    ax.set_xlabel("")
        
    ax.legend(fontsize=35, ncol=3)
    ax.set_ylabel("$Centered <P(%s|stage)>$"%kind[1], fontsize=35)
    fig.savefig(f"topic_{topic}.pdf")
    

    plt.show()

In [ ]:
for topic in df_topic_stage_gb.columns:
    fig = plt.figure()
    nulls = []
    for _ in range(1000):
        df_temp = df_topics[mask].copy()
        stages = df_temp["stage"].values
        np.random.shuffle(stages)
        df_temp["stage"] = stages
        y = df_temp.groupby(["stage"]).mean()[topic]
        x = np.arange(0,len(y))
        nulls.append(pearsonr(x,y)[0])
        
    df_topic_stage_dat = df_topic_stage_gb[topic]
    mean = df_topic_stage_dat.mean()
    df_topic_stage_dat=df_topic_stage_dat.subtract(mean,1)
    #sigma = df_topic_stage_dat.std(0).values[0]
    #df_topic_stage_dat=df_topic_stage_dat.divide(sigma,1)
    x, y = np.arange(0,len(df_topic_stage_dat)), df_topic_stage_dat
    
    data_corr = pearsonr(x,y)[0]
    
    bins = np.linspace(-1.05, 1.05, 21)
        
    integral_mask = hist[1][:-1]>data_corr
    
    P_corr = (np.array(nulls)>data_corr).sum()/len(nulls)
    
    
    plt.hist(nulls, bins = bins, color="gray", lw=5, histtype="step", density=True)
    plt.vlines(data_corr, 0, 0.8, ls="--", colors="red")
    plt.xlabel("Correlation")
    plt.ylabel("pdf")
    Z = np.abs(np.average(nulls)-data_corr)/np.std(nulls)
    plt.title("Z: {:.3f} \n P_corr: {:.3f} \n P_Anticorr: {:.3f}".format(Z, P_corr, 1-P_corr))
    fig.savefig(f"topic_{topic}_pval.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(9,8))
from topicpy.hsbmpy import color_iterator
for metadatum in df_topics.columns[1:]:
    color = next(color_iterator)
    subdf = df_topics.loc[:, ["stage", metadatum]].replace("",np.nan).replace("nan",np.nan).dropna().sort_values("stage")
    ax.scatter(subdf["stage"], subdf[metadatum], label=metadatum, color=color)
    ax.plot(subdf["stage"].unique(), subdf.groupby("stage").median()[metadatum], marker="x", color=color)

ax.set_ylabel("P(sample|metadato)")
ax.legend()

In [ ]:
def get_cohort(subtype):
    if str(subtype)=="nan":
        return None
    else:
        if "Basal" in subtype:
            return "A"
        else:
            return "B"
df_files["cohort"]=list(map(get_cohort,df_files["Subtype_Selected"]))

In [ ]:
df_files = df_files.set_index("cases.0.submitter_id")
mask = (~df_files["days_survival"].isna()) & (df_files["cases.0.diagnoses.0.tumor_stage"]!="not reported")
subset = df_files[mask]
df_files=df_files.reset_index()
subset = subset[["days_survival","vital_status","gender", "tumor_stage", "age_at_diagnosis"]]
for topic in df_clusters.columns:
    top_set = add_group_to_subset(topic, subset, df_clusters.divide(df_clusters.sum(0),1), 0.35)
    print(top_set["group"].sum())
    summary, _, ax = fit_cox(top_set, topic)
    if summary is not None:
        if summary.at[summary.index[-1],"-log2(p)"]/(np.log2(10)) > 2:
            print(dataset,": ",topic,"\n",summary.loc[summary.index[-1],["coef", "p"]],"\n")
            ax=None
            if ax is not None:
                ax.set_title(dataset+" "+ax.title.get_text(), fontsize=35)
                save_plot(ax, dataset, topic)

## Multivariate

In [ ]:
from lifelines import CoxPHFitter
from lifelines.datasets import load_rossi

In [ ]:
algorithm = "trisbm"
l = 1

In [ ]:
df_covariates = pd.DataFrame(index=df_files["cases.0.submitter_id"])
df_covariates["vital_status"] = df_files["vital_status"].values.astype(int)
df_covariates["time"] = df_files["days_survival"].values

In [ ]:
df_topics = pd.read_csv("%s/%s_level_%d_topic-dist.csv"%(algorithm,algorithm,l), index_col=1).drop("i_doc",1)
df_topics_std = df_topics.divide(df_topics.sum(0),1)

q = np.linspace(0, 1, 4)

for topic in df_topics:
    quantiles = np.argmin([np.abs(df_topics_std[topic]-q) for q in df_topics_std[topic].quantile(q).values], 0)
    df_covariates = df_covariates.join(pd.Series(name=topic, index = df_topics_std.index, data = quantiles), how="outer")

In [ ]:
cph = CoxPHFitter()
res = cph.fit(df_covariates.dropna(how="any", axis=0), duration_col='time', event_col='vital_status')

cph.print_summary()
cph.plot()

In [ ]:
df_metadata = pd.read_csv("%s/%s_level_%d_metadatum-dist.csv"%(algorithm,algorithm,l), index_col=1).drop("i_doc",1)
df_metadata_std = df_metadata.subtract(df_metadata.mean(0),1)
for topic in df_metadata:
    quantiles = np.argmin([np.abs(df_metadata_std[topic]-q) for q in df_metadata_std[topic].quantile(q).values], 0)
    df_covariates = df_covariates.join(pd.Series(name=topic, index=df_covariates.index, data = quantiles), how="outer")

In [ ]:
cph_full = CoxPHFitter()
res_full = cph_full.fit(df_covariates.dropna(how="any", axis=0), duration_col='time', event_col='vital_status')

cph_full.print_summary()
cph_full.plot()

In [ ]:
fig,ax = plt.subplots()

params = {"ax":ax, "histtype":"step", "bins":np.linspace(0, 2, 15), "density":False, "lw":5, "ls":":"}

res.summary["exp(coef)"].hist(**params)
res_full.summary["exp(coef)"].hist(**params)
#ax.vlines(1, 0, 5, lw=5)
ax.set_xlabel("exp(coef)")
ax.set_ylabel("dimensions (topic+metadata)")

In [ ]:
for topic in df_topics:
    fig, axs = plt.subplots(1,2, figsize=(18,5))
    axs[0].set_ylabel("Survival")
    axs[0].set_xlabel("Days")
    axs[1].set_xlabel("Days")
    cph.plot_partial_effects_on_outcome(covariates=[topic], 
                                        values = np.sort(df_covariates["Metadatum 3"].unique()),
                                        cmap = 'coolwarm', ax=axs[0])
    cph_full.plot_partial_effects_on_outcome(covariates=topic, values=np.sort(df_covariates[topic].unique()), cmap='coolwarm', ax=axs[1])